## <center> COVID-19 TRACKING (Statistics and Graphs )

Pour realiser ce tuto, il faut tout d'abord accéder aux données du coronavirus à partir du lien : <font size=3>https://datahub.io/core/covid-19/r/time-series-19-covid-combined.csv </font> 

In [1]:
from datetime import datetime
from ipywidgets import interact_manual, interact,interactive
import ipywidgets as ipw
import pandas as pd
import plotly.graph_objects as graph

In [2]:
df=pd.read_csv('time-series-19-covid-combined.csv')

In [3]:
df.head()

,Date,Country/Region,Province/State,Lat,Long,Confirmed,Recovered,Deaths
0,2020-01-22,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0
1,2020-01-23,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0
2,2020-01-24,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0
3,2020-01-25,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0
4,2020-01-26,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0


In [4]:
df['Date'].head()

0    2020-01-22
1    2020-01-23
2    2020-01-24
3    2020-01-25
4    2020-01-26
Name: Date, dtype: object

#### Nos données contiennent des dates mais dans un objet chaîne, elles seront donc transformées en un objet datetime afin de les tracer 

In [5]:
def convert_to_date(value):
    return datetime.strptime(value, '%Y-%m-%d')
df['Date'] = df['Date'].apply(convert_to_date)

In [6]:
df['Date'].head()

0   2020-01-22
1   2020-01-23
2   2020-01-24
3   2020-01-25
4   2020-01-26
Name: Date, dtype: datetime64[ns]

#### Nous regrouperons les données par pays

In [7]:
def groupDataBycountry(country):
    gk=df.groupby('Country/Region')
    final_dataset=(gk.get_group(country))[['Date','Confirmed','Recovered','Deaths']]
    return final_dataset.set_index('Date')

#### Ensuite, on fait l'extraction des données groupées le nombre de cas touchés, le nombre de décès et le nombre de cas récupérés d'aujourd'hui

In [8]:
def extract_last_case(country):
    data=groupDataBycountry(country)
    d=data.iloc[-1]
    return (data.index[-1],d['Confirmed'],d['Recovered'],d['Deaths'])

#### Maintenant, nous allons définir des fonctions qui tracent les trois cas

In [9]:
def plot_confirmed_cases(fig,data):
    fig.add_trace(graph.Scatter(
            x=data.index, y=data.Confirmed ,  name='Confirmé',fill="tozeroy",
            marker_color='#77b5fe'
        ))

In [10]:
def plot_recovered_cases(fig,data):
    fig.add_trace(graph.Scatter(
            x=data.index, y=data.Recovered ,  name='Rétabli',fill="tozeroy",
            marker_color='rgb(63, 226, 101)'
        ))

In [11]:
def plot_deaths(fig,data):
    fig.add_trace(graph.Scatter(
            x=data.index, y=data.Deaths ,  name='Morts',fill="tozeroy",
            marker_color='rgb(255, 153, 153)'
        ))

#### la fonction ci-dessous appliquera le bon thème à notre graph (clair / foncé)

In [12]:
def apply_theme(fig,country,color_theme,font_color):
    fig.layout.update({
    'plot_bgcolor': color_theme,
    'paper_bgcolor': 'rgba(0,0,0,0)',
    })
    fig.layout.update(
        title={
        'text': "COVID-19 Evolution dans "+country,
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        xaxis_title="Date",
        yaxis_title="Cas",
        legend=dict(
            x=0.01,
            y=1,
            traceorder="normal",
            font=dict(
                family="sans-serif",
                size=12,
                color=font_color
            ),
            bordercolor=color_theme,
            borderwidth=1
        ))
    fig.update_xaxes(showline=True, linewidth=1, linecolor='gray')
    fig.update_yaxes(showline=True, linewidth=1, linecolor='gray')

#### la fonction qui trace le graphique en fonction du pays choisi

In [13]:
def plot_graph(data,country,confirmed,recovered,deaths,theme):
    fig=graph.Figure()
    if(confirmed==True):
        plot_confirmed_cases(fig,data)
    if(recovered==True):
        plot_recovered_cases(fig,data)
    if(deaths==True):
        plot_deaths(fig,data)
    if(theme=="Foncé"):
        color_theme="rgb(50,50,50)"
        font_color="white"
    else:
        color_theme="rgba(0,0,0,0)"
        font_color="black"
    apply_theme(fig,country,color_theme,font_color)
    fig.show()

#### Ensuite, nous définirons les widgets utilisés

In [14]:
dropdown=ipw.Dropdown(options=list(dict.fromkeys(df['Country/Region'].tolist())),description="Pays :")
deaths=ipw.Checkbox( 
    value=False,
    description='Morts',
    disabled=False
)
recovered=ipw.Checkbox( 
    value=False,
    description='Rétabli',
    disabled=False
)
confirmed=ipw.Checkbox( 
    value=True,
    description='Confirmé',
    disabled=False
)
theme=ipw.RadioButtons(
    options=['Clair', 'Foncé'],
    description="theme :",
    value='Clair',
    disabled=False
)

####  la fonction exécuter appelle toutes les autres fonctions écrites ci-dessus

In [15]:
global Title
global Title2

In [16]:
def execute(country,confirmed,recovered,deaths,theme):
    data=groupDataBycountry(country)
    v=extract_last_case(country)
    Title=ipw.HTML(value="<center><h3> Evolution de <font color='#07CEF5'>COVID-19</font> dans le pays <font color='blue'>"+country+"</font></h3></center>")
    Title2=ipw.HTML(value="<center><h4><font >"+str(v[0])+" </font>  <font color='blue'>Confirmé : </font>"+str(v[1])+" <font color='green'>Rétabli : </font>"+str(v[2])+" <font color='red'>Morts : </font>"+str(v[3])+"</h4></center>")
    display(Title,Title2)
    return plot_graph(data,country,confirmed,recovered,deaths,theme)

#### Finalement, nous écrivons la fonction qui sera appelée chaque fois que nous interagirons avec les widgets

In [17]:
Title1=ipw.HTML(value="<center><h4><font color='red'>Par ESSADIK Ibtissam</font>")    
display(ipw.HTML(value="<U><center><h1>Choisissez <font color='#BA48E1'> le pays</font> ,<font color='#F2B603'>les cas </font> que vous souhaitez voir et le <font color='#049393'>thème</font></center> </U>"),Title1)
def final(a,b,c,d,e):
    execute(dropdown.value,confirmed.value,recovered.value,deaths.value,theme.value)
w=interactive(final,a=dropdown,b=deaths,c=recovered,d=confirmed,e=theme)
w

HTML(value="<U><center><h1>Choisissez <font color='#BA48E1'> le pays</font> ,<font color='#F2B603'>les cas </f…

HTML(value="<center><h4><font color='red'>Par ESSADIK Ibtissam</font>")

interactive(children=(Dropdown(description='Pays :', options=('Afghanistan', 'Albania', 'Algeria', 'Andorra', …